* [Policy Evaluation](#Policy-Evaluation)
* [Policy Iteration](#Policy-Iteration)
* [Value Iteration](#Value-Iteration)

In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2
from gridworld_utils import print_grid_values, print_grid_policy

### Gridworld
For all gridworlds in the following examples:
* There are 4 actions - left, up, right, down
* The environment is deterministic - taking some action from some state will always lead to the same successor state
* Moving into the edge will cause you to stay where you are
* Each move/step results in a reward of $-1$
* Terminal states have a constant value of $0$

# Policy Evaluation

Evaluating the exact state-values for a given policy in some gridworld.

In [2]:
def update_state_value(policy, state_values, reward=-1, discount=1):
    v = 0
    # Bellman Expectation Equation for deterministic environment. (Taking
    # some action in some state will always lead to the same successor state).
    for p, s_v in zip(policy, state_values):
        v += p * (reward + discount * s_v)
    return v

In [3]:
def one_sweep_policy_evaluation(grid, policy, terminal_states):
    rows, columns = grid.shape
    new_grid = np.zeros(grid.shape)
    for i in range(rows):
        for j in range(columns):
            
            # Making sure not to update terminal states
            is_terminal_state = False
            for terminal_state in terminal_states:
                x, y = terminal_state
                if i == x and j == y:
                    is_terminal_state = True
            if is_terminal_state:
                continue
                
            # Finding previous state values for all possible succussor states 
            v_left = grid[i][j] if j == 0 else grid[i][j-1]
            v_up = grid[i][j] if i == 0 else grid[i-1][j]
            v_right = grid[i][j] if j == (columns-1) else grid[i][j+1]
            v_down = grid[i][j] if i == (rows-1) else grid[i+1][j]
            
            new_grid[i][j] = update_state_value(policy[i][j], (v_left,v_up,v_right,v_down))
    return new_grid
    

In [4]:
def full_policy_evaluation(grid, policy, terminal_states, theta=0.01):
    difference = 100 # Arbitrary large initial difference 
    iteration = 0 
    
    while difference > theta: # theta controls how accurately we continue to evaluate for
        previous_grid = np.copy(grid)
        grid = one_sweep_policy_evaluation(grid, policy, terminal_states)
        difference = np.max(np.abs(previous_grid - grid))
        iteration += 1
    return grid, iteration

In [5]:
def initialise_random_policy(shape, terminal_states):
    policy = np.empty(shape, dtype=object)
    policy.fill((1/4, 1/4, 1/4, 1/4))
    for terminal_state in terminal_states:
        policy[terminal_state] = None
    return policy

### Examples

This first example is a 4x4 gridworld with two corner terminal states, like the example in chapter 4.1 of Sutton and Barto's 'Rienforcement Learning: An Introduction'.

In [6]:
gridworld_4x4 = np.zeros((4,4))
terminal_states = ((0,0), (3,3)) # Top left and bottom right corners
policy = initialise_random_policy(gridworld_4x4.shape, terminal_states)

for k in range(4):
    print('State-values at iteration {}:'.format(k))
    print_grid_values(gridworld_4x4)
    gridworld_4x4 = one_sweep_policy_evaluation(gridworld_4x4, policy, terminal_states)
    

State-values at iteration 0:
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|

State-values at iteration 1:
|=======|=======|=======|=======|
|  0.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  |  0.0  |
|=======|=======|=======|=======|

State-values at iteration 2:
|=======|=======|=======|=======|
|  0.0  | -1.8  | -2.0  | -2.0  |
|=======|=======|=======|=======|
| -1.8  | -2.0  | -2.0  | -2.0  |
|=======|=======|=======|=======|
| -2.0  | -2.0  | -2.0  | -1.8  |
|=======|=======|=======|=======|
| -2.0  | -2.0  | -1.8  |  0.0  |
|=======|=======|=======|==

In [7]:
gridworld_6x6 = np.zeros((6,6))
terminal_states = ((2,2), (2,3), (3,2), (3,3)) # The middle 4 squares
policy = initialise_random_policy(gridworld_6x6.shape, terminal_states)
theta = 0.001 

gridworld_6x6, k = full_policy_evaluation(gridworld_6x6, policy, terminal_states, theta)
print('State-values after iteration {}:'.format(k))
print_grid_values(gridworld_6x6)


State-values after iteration 159:
|=======|=======|=======|=======|=======|=======|
| -27.0 | -25.0 | -22.5 | -22.5 | -25.0 | -27.0 |
|=======|=======|=======|=======|=======|=======|
| -25.0 | -21.5 | -16.0 | -16.0 | -21.5 | -25.0 |
|=======|=======|=======|=======|=======|=======|
| -22.5 | -16.0 |  0.0  |  0.0  | -16.0 | -22.5 |
|=======|=======|=======|=======|=======|=======|
| -22.5 | -16.0 |  0.0  |  0.0  | -16.0 | -22.5 |
|=======|=======|=======|=======|=======|=======|
| -25.0 | -21.5 | -16.0 | -16.0 | -21.5 | -25.0 |
|=======|=======|=======|=======|=======|=======|
| -27.0 | -25.0 | -22.5 | -22.5 | -25.0 | -27.0 |
|=======|=======|=======|=======|=======|=======|



# Policy Iteration

Whats happeing:
1. Beginning with a random policy and all state-values set to 0
2. Evaluating that policy all the way untill changes being made are less than set parameter theta (**Policy Evaluation**)
3. Acting greedily on the values for that policy to generate a better policy (**Policy Improvement**)
4. Repeating steps 2 and 3 untill the improved policies are equal to each other and thus equal to the optimal policy (**Policy Iteration**)

In [8]:
# Getting an improved Policy by taking greedy actions with respect to the
# immediate reward + value of next state
def get_greedy_policy(grid, terminal_states, reward=-1, discount=1):
    rows, columns = grid.shape
    policy = np.empty(grid.shape, dtype=object)
    for i in range(rows):
        for j in range(columns):
            
            # Making sure not to update terminal states
            is_terminal_state = False
            for terminal_state in terminal_states:
                x, y = terminal_state
                if i == x and j == y:
                    is_terminal_state = True
            if is_terminal_state:
                continue
                
            # Finding previous state values for all possible succussor states 
            v_left = grid[i][j] if j == 0 else grid[i][j-1]
            v_up = grid[i][j] if i == 0 else grid[i-1][j]
            v_right = grid[i][j] if j == (columns-1) else grid[i][j+1]
            v_down = grid[i][j] if i == (rows-1) else grid[i+1][j]
            
            # Finding the the values of every action from current state and
            # acting greedly 
            successor_state_values = np.array([v_left, v_up, v_right, v_down])
            action_values = reward + discount * successor_state_values
            greedy_actions = action_values == np.max(action_values)
            policy[i][j] = tuple(greedy_actions / np.count_nonzero(greedy_actions))
    return policy
            

In [9]:
def policy_iteration(grid, policy, terminal_states, theta):
    print('Initial state-values and policy:')
    print_grid_values(grid)
    print_grid_policy(policy)

    improved_policies = 0
    total_k = 0
    while True:
        grid, k = full_policy_evaluation(grid, policy, terminal_states, theta)
        total_k += k
        
        previous_policy = np.copy(policy)
        policy = get_greedy_policy(grid, terminal_states)
        improved_policies += 1
        
        if (previous_policy == policy).all():
            print('Optimal policy found!')
            return grid, policy
        
        print('State-values and improved policy #{} (k={}):'.format(improved_policies, total_k))
        print_grid_values(grid)
        print_grid_policy(policy)
        
        

### Examples

In [10]:
gridworld_4x4 = np.zeros((4,4))
terminal_states = ((0,0), (3,3)) # Top left and bottom right corners
policy = initialise_random_policy(gridworld_4x4.shape, terminal_states)

print('Initial state-values and policy:')
print_grid_values(gridworld_4x4)
print_grid_policy(policy)

for k in range(4):
    gridworld_4x4 = one_sweep_policy_evaluation(gridworld_4x4, policy, terminal_states)
print('State-values and greedy policy w.r.t. v after 3 iterations:')
print_grid_values(gridworld_4x4)
improved_policy = get_greedy_policy(gridworld_4x4, terminal_states)
print_grid_policy(improved_policy)

Initial state-values and policy:
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|

|=======|=======|=======|=======|
|///////| ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  |///////|
|=======|=======|=======|=======|

State-values and greedy policy w.r.t. v after 3 iterations:
|=======|=======|=======|=======|
|  0.0  | -3.1  | -3.8  | -4.0  |
|=======|=======|=======|=======|
| -3.1  | -3.7  | -3.9  | -3.8  |
|=======|=======|=======|=======|
| -3.8  | -3.9  | -3.7  | -3.1  |
|=======|=======|=======|=======|
| -4.0  | -3.8  | -3.1  |  0.0  |
|=======|=======|====

In [11]:
gridworld_8x8 = np.zeros((8,8))
terminal_states = ((2, 2), (3, 2), (4, 2), (4, 3), (6, 6), (7, 0))
policy = initialise_random_policy(gridworld_8x8.shape, terminal_states)
theta = 0.01

v, pi = policy_iteration(gridworld_8x8, policy, terminal_states, theta)

Initial state-values and policy:
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|==

# Value Iteration

Value Iteration is simply always updating with the values with respect to the action that leads to the highest immediate reward + value of, in this case, the previous iteration (synchronous update). This can be achieved by performing Policy Iteration with only one sweep of updates to the values before choosing the new greedy policy and repeating. Concisely, you are just iteratively updating each state-value with respect to the Bellman Optimality Equation:

$$\mathcal{v}_{k+1}(s) = \max_a \sum_{s', r} p(s', r \mid s, a) [r + \gamma \mathcal{v}_k(s')]$$

In [12]:
def value_iteration(grid, terminal_states, theta=0.01, reward=-1, discount=1):
    rows, columns = grid.shape
    iteration = 0
    
    # Looping through value iterations until updates beting made
    # are less than theta.
    while True:
        iteration += 1
        pre_grid = np.copy(grid)
        # Looping through every single state and updating its value.
        for i in range(rows):
            for j in range(columns):
                
                # Making sure not to update terminal states
                is_terminal_state = False
                for terminal_state in terminal_states:
                    x, y = terminal_state
                    if i == x and j == y:
                        is_terminal_state = True
                if is_terminal_state:
                    continue

                # Finding previous state values for all possible succussor states 
                v_left = pre_grid[i][j] if j == 0 else pre_grid[i][j-1]
                v_up = pre_grid[i][j] if i == 0 else pre_grid[i-1][j]
                v_right = pre_grid[i][j] if j == (columns-1) else pre_grid[i][j+1]
                v_down = pre_grid[i][j] if i == (rows-1) else pre_grid[i+1][j]

                # Bellman Optimality Equation (in this gridworld the probability
                # of being in a state, s, and taking action, a, and getting a 
                # reward, r, and ending up in state, s', is always 100%).
                successor_state_values = np.array([v_left, v_up, v_right, v_down])
                action_values = reward + discount * successor_state_values
                grid[i][j] = np.max(action_values)
        
        difference = np.max(np.abs(pre_grid - grid))
        if difference < theta:
            policy = get_greedy_policy(grid, terminal_states)
            print('≈ Optimal value function and policy found ({} iterations):'.format(iteration))
            print_grid_values(grid)
            print_grid_policy(policy)
            return grid, policy

In [13]:
gridworld_8x8 = np.zeros((8,8))
terminal_states = ((2, 2), (3, 2), (4, 2), (4, 3), (6, 6), (7, 0))
policy = initialise_random_policy(gridworld_8x8.shape, terminal_states)
theta = 0.01

v, pi = value_iteration(gridworld_8x8, terminal_states, theta)

≈ Optimal value function and policy found (8 iterations):
|=======|=======|=======|=======|=======|=======|=======|=======|
| -4.0  | -3.0  | -2.0  | -3.0  | -4.0  | -5.0  | -6.0  | -7.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -3.0  | -2.0  | -1.0  | -2.0  | -3.0  | -4.0  | -5.0  | -6.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -2.0  | -1.0  |  0.0  | -1.0  | -2.0  | -3.0  | -4.0  | -5.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -2.0  | -1.0  |  0.0  | -1.0  | -2.0  | -3.0  | -3.0  | -4.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -2.0  | -1.0  |  0.0  |  0.0  | -1.0  | -2.0  | -2.0  | -3.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -2.0  | -2.0  | -1.0  | -1.0  | -2.0  | -2.0  | -1.0  | -2.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
| -1.0  | -2.0  | -2.0  | -2.0  | -2.0  | -1.0  |  0.0  | -1.0  |
|=======|=======|=

Note -- with the same theta only 8 state-value updates were required in Value Iteration compared to 184 in Policy iteration.

**Change the hyperparameters above to create a unique gridworld and find the optimal value function and policy** 